In [1]:
from warnings import filterwarnings

filterwarnings("ignore")

from pybats.analysis import analysis
from pybats.point_forecast import median
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from random import randint

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score


from quant_invest_lab.data_provider import download_crypto_historical_data
from quant_invest_lab.backtest import ohlc_long_only_backtester

In [2]:
def measure_performances(y_true: np.ndarray, y_pred: np.ndarray, comment: str) -> None:
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    r2 = r2_score(y_true, y_pred)
    print(f"{comment}: MAE={mae:.2f}, RMSE={rmse:.2f}, R2={r2:.2f}")

In [6]:
symbol = "BTC-USDT"
timeframe = "4hour"
df_BTC = download_crypto_historical_data(symbol, timeframe)  # .iloc[-3000:]
df_BTC["EMA20"] = df_BTC.Close.ewm(span=20).mean()
df_BTC.dropna(inplace=True)
print(df_BTC.shape)
df_BTC.head()

(12200, 9)


,Timestamp,Open,Close,High,Low,Amount,Volume,Returns,EMA20
Date,,,,,,,,,
2017-10-18 18:00:00,1.508342e+09,3996.866017,3811.101002,4318.732702,3806.381676,0.120964,467.067707,0.000000,3811.101002
2017-10-18 22:00:00,1.508357e+09,3811.101001,3812.004238,4088.280609,3811.101000,0.062151,241.111516,0.000237,3811.575201
2017-10-19 02:00:00,1.508371e+09,3812.004225,4060.402955,5548.231409,3812.000000,0.136836,545.171668,0.065162,3902.943260
2017-10-19 06:00:00,1.508386e+09,4060.021494,5123.413913,5693.210514,3806.381676,0.375341,1647.890008,0.261799,4255.274268
2017-10-19 10:00:00,1.508400e+09,5093.210514,5093.210517,5693.210514,5093.210514,0.930882,5071.213105,-0.005895,4457.963916


In [7]:
Y = df_BTC.Close.dropna().values
k = 1  # forecasting one step ahead
forecast_start = 0  # starting forecast at time step 0
forecast_end = len(df_BTC) - 1  # ending forecast at the same time our data ends

mod, samples = analysis(
    Y,
    family="poisson",  # the family of the distribution to be used
    forecast_start=forecast_start,
    forecast_end=forecast_end,
    k=k,
    nsamps=100,  # number of samples we draw for each month
    prior_length=100,  # number of points that define the prior distribution
    rho=0.9,  # random effect extension
    deltrend=0.5,  # discount factor for trend component
    delregn=0.9,  # discount factor for regression component
)

forecast = median(samples)
measure_performances(df_BTC.Close.values, forecast, "poisson")
df_BTC["Forecast"] = forecast

poisson: MAE=301.39, RMSE=737.70, R2=1.00


In [8]:
ind = randint(1, len(df_BTC) - 100)

df_BTC_short = df_BTC.iloc[ind : ind + 100]
fig = make_subplots(
    rows=1,
    cols=1,
    subplot_titles=("Historical price forecasted"),
    shared_xaxes=True,
)

fig.add_trace(
    go.Candlestick(
        name="Historical price",
        x=df_BTC_short.index,
        open=df_BTC_short["Open"],
        high=df_BTC_short["High"],
        low=df_BTC_short["Low"],
        close=df_BTC_short["Close"],
    ),
    row=1,
    col=1,
)

fig.add_trace(
    go.Scatter(
        name="Forecast",
        x=df_BTC_short.index,
        y=df_BTC_short["Forecast"],
    ),
    row=1,
    col=1,
)
fig.add_trace(
    go.Scatter(
        name="EMA20",
        x=df_BTC_short.index,
        y=df_BTC_short["EMA20"],
    ),
    row=1,
    col=1,
)
fig.update_layout(
    xaxis_rangeslider_visible=False,
    showlegend=True,
    title_text="Historical price and forecasted price",
)
fig.show()

# Online forecasting


In [9]:
symbol = "BTC-USDT"

df_BTC = download_crypto_historical_data(symbol, timeframe)  # .iloc[-3000:]
df_BTC["EMA20"] = df_BTC.Close.ewm(span=20).mean()
df_BTC.dropna(inplace=True)
print(df_BTC.shape)
df_BTC.head()

(12200, 9)


,Timestamp,Open,Close,High,Low,Amount,Volume,Returns,EMA20
Date,,,,,,,,,
2017-10-18 18:00:00,1.508342e+09,3996.866017,3811.101002,4318.732702,3806.381676,0.120964,467.067707,0.000000,3811.101002
2017-10-18 22:00:00,1.508357e+09,3811.101001,3812.004238,4088.280609,3811.101000,0.062151,241.111516,0.000237,3811.575201
2017-10-19 02:00:00,1.508371e+09,3812.004225,4060.402955,5548.231409,3812.000000,0.136836,545.171668,0.065162,3902.943260
2017-10-19 06:00:00,1.508386e+09,4060.021494,5123.413913,5693.210514,3806.381676,0.375341,1647.890008,0.261799,4255.274268
2017-10-19 10:00:00,1.508400e+09,5093.210514,5093.210517,5693.210514,5093.210514,0.930882,5071.213105,-0.005895,4457.963916


In [10]:
def online_prediction(Y_close: pd.Series):
    Y = Y_close.values
    k = 1  # forecasting one step ahead
    forecast_start = 0  # starting forecast at time step 0
    forecast_end = len(Y) - 1  # ending forecast at the same time our data ends

    mod, samples = analysis(
        Y,
        family="poisson",  # the family of the distribution to be used
        forecast_start=forecast_start,
        forecast_end=forecast_end,
        k=k,
        nsamps=100,  # number of samples we draw for each month
        prior_length=10,  # number of points that define the prior distribution
        rho=0.9,  # random effect extension
        deltrend=0.5,  # discount factor for trend component
        delregn=0.9,  # discount factor for regression component
    )
    return median(samples)[-1]


window = 600
df_BTC["Online_forecast"] = df_BTC.Close.rolling(window).apply(online_prediction)
df_BTC.dropna(inplace=True)
df_BTC["Online_forecast"]

In [ ]:
ind = randint(1, len(df_BTC) - 100 - window)

df_BTC_short = df_BTC.dropna().iloc[ind : ind + 100]

fig = make_subplots(
    rows=1,
    cols=1,
    subplot_titles=("Historical price online-forecasted"),
    shared_xaxes=True,
)

fig.add_trace(
    go.Candlestick(
        name="Historical price",
        x=df_BTC_short.index,
        open=df_BTC_short["Open"],
        high=df_BTC_short["High"],
        low=df_BTC_short["Low"],
        close=df_BTC_short["Close"],
    ),
    row=1,
    col=1,
)

fig.add_trace(
    go.Scatter(
        name="Online-forecast",
        x=df_BTC_short.index,
        y=df_BTC_short["Online_forecast"],
    ),
    row=1,
    col=1,
)
fig.update_layout(
    xaxis_rangeslider_visible=False,
    showlegend=True,
    title_text="Historical price and forecasted price",
)
fig.show()

In [ ]:
def buy_func(row, prev_row) -> bool:
    return (
        True
        if row["Online_forecast"] < row["Close"]
        and prev_row["Online_forecast"] > prev_row["Close"]
        # and row["Online_forecast"] > row["EMA20"]
        else False
    )


def sell_func(row, prev_row, trading_days) -> bool:
    return (
        True
        if row["Online_forecast"] > row["Close"]
        and prev_row["Online_forecast"] < prev_row["Close"]
        # and row["Online_forecast"] < row["EMA20"]
        else False
    )


d = ohlc_long_only_backtester(
    df_BTC, buy_func, sell_func, timeframe=timeframe, get_trade_df=True
)